# Lang chain RAG

El comando instala y actualiza las librerías langchain, langchain-community, y langchain-chroma en un entorno de Jupyter Notebook. Estas herramientas se usan para crear aplicaciones con modelos de lenguaje y optimizar búsquedas. La opción --quiet reduce mensajes, y --upgrade asegura que se instale la última versión.

In [2]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\sebas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


El código configura dos variables de entorno: activa el seguimiento en LangChain (LANGCHAIN_TRACING_V2) y solicita al usuario ingresar su clave API (LANGCHAIN_API_KEY) de forma segura con getpass. Esto permite rastrear y acceder a funciones avanzadas de LangChain en el entorno.

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

Este código configura una clave de API de OpenAI y crea una instancia de un modelo de lenguaje (ChatOpenAI) de LangChain para interactuar con el modelo gpt-4o-mini. La clave de API (OPENAI_API_KEY) se establece directamente en el entorno, permitiendo el acceso a las funcionalidades de OpenAI.

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Este comando instala el paquete beautifulsoup4 en el entorno actual. Beautiful Soup es una biblioteca de Python utilizada para analizar y extraer datos de archivos HTML y XML, facilitando el web scraping o la manipulación de datos en estos formatos.

In [5]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\sebas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Este código carga el contenido de un blog, lo divide en fragmentos de texto y lo indexa usando Chroma y OpenAIEmbeddings para facilitar la búsqueda. Luego, configura una cadena para responder preguntas sobre el contenido usando LangChain. Finalmente, se consulta el índice con la pregunta: "What is Task Decomposition?".

In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps. It often involves using techniques like Chain of Thought (CoT) or Tree of Thoughts to facilitate reasoning and planning. This approach allows an agent to tackle intricate tasks systematically and efficiently.'

El método vectorstore.delete_collection() elimina toda la colección de vectores almacenada en vectorstore, lo que borra el índice y los datos asociados. Esto es útil para liberar espacio o reiniciar el índice antes de cargar nuevos documentos.

In [7]:
vectorstore.delete_collection()

# Detailed walkthrough

## Indexing: Load

Este código carga contenido específico de una página web (título, encabezado y contenido) usando BeautifulSoup y WebBaseLoader. Se enfoca en extraer solo ciertas secciones de la página web mediante SoupStrainer, filtrando la HTML para obtener únicamente las clases especificadas: post-title, post-header y post-content.

Finalmente, el código devuelve la longitud del contenido del primer documento cargado (docs[0].page_content), mostrando la cantidad de caracteres en ese fragmento de texto.

In [8]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

El código print(docs[0].page_content[:500]) imprime los primeros 500 caracteres del contenido extraído del primer documento cargado. Esto es útil para visualizar una muestra del texto y verificar que la carga del contenido fue exitosa.

Si el contenido tiene más de 500 caracteres, solo se mostrará una parte inicial; si tiene menos, se imprimirá todo el texto disponible.

In [9]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## Indexing: Split

Este código utiliza RecursiveCharacterTextSplitter para dividir el contenido del documento cargado (docs) en fragmentos (chunks) de texto. Configura un tamaño de fragmento de 1000 caracteres y una superposición de 200 caracteres entre fragmentos. Además, con la opción add_start_index=True, se agrega un índice de inicio a cada fragmento.

Finalmente, el código devuelve el número de fragmentos generados por la división del documento, mediante len(all_splits). Esto te muestra cuántos fragmentos resultaron del proceso de segmentación.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

El código len(all_splits[0].page_content) devuelve la longitud (en caracteres) del contenido del primer fragmento (all_splits[0]) generado por el RecursiveCharacterTextSplitter. Esto permite verificar cuántos caracteres tiene el primer fragmento de texto después de la división, y asegurarse de que cumple con el tamaño configurado de 1000 caracteres, teniendo en cuenta la superposición de 200 caracteres.

In [11]:
len(all_splits[0].page_content)

969

El código all_splits[10].metadata accede a los metadatos del décimo fragmento (all_splits[10]) generado por el RecursiveCharacterTextSplitter. Los metadatos contienen información adicional sobre ese fragmento, como su posición en el texto original, el índice de inicio, el número de documento original, o cualquier otra información relevante que pueda haber sido incluida durante el proceso de segmentación.

Esta información es útil para rastrear la procedencia de cada fragmento o para realizar análisis adicionales sobre la estructura de los datos procesados.

In [12]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

## Indexing: Store

El código crea un índice de vectores usando Chroma para almacenar los fragmentos de texto (all_splits). Utiliza OpenAIEmbeddings para convertir cada fragmento en una representación vectorial. El resultado es un vectorstore que permite realizar búsquedas semánticas rápidas en los documentos indexados.

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

## Retrieval and Generation: Retrieve

Este código configura un retriever que realiza búsquedas de similitud en el vectorstore. El parámetro search_type="similarity" indica que la búsqueda se realiza según la similitud semántica entre la consulta y los fragmentos almacenados. La opción search_kwargs={"k": 6} limita el número de documentos recuperados a los 6 más similares.

Luego, la línea retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?") ejecuta la búsqueda con la pregunta proporcionada. Finalmente, len(retrieved_docs) devuelve el número de documentos recuperados por el retriever, que en este caso debería ser 6 (si se encuentran suficientes coincidencias).

In [14]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

6

El código print(retrieved_docs[0].page_content) imprime el contenido del primer documento recuperado por el retriever. El retrieved_docs[0] corresponde al primer fragmento de texto que el sistema ha encontrado como el más relevante o similar a la consulta realizada ("What are the approaches to Task Decomposition?").

Esto te permite ver la información contenida en ese fragmento específico, que es el más relacionado con la pregunta planteada.

In [15]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


## Retrieval and Generation: Generate

comando pip install -qU langchain-openai actualiza e instala de manera silenciosa (-q para "quiet") la última versión de la librería langchain-openai. Esta biblioteca proporciona integración con los modelos de OpenAI en el ecosistema de LangChain, permitiendo usar modelos de lenguaje como GPT para tareas de procesamiento de texto, generación de contenido, y más. La opción -U asegura que se instale la versión más reciente disponible.

In [16]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\sebas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



El código solicita de manera segura la clave de API de OpenAI usando getpass y la configura como una variable de entorno. Luego, importa y utiliza ChatOpenAI para crear una instancia del modelo gpt-4o-mini de OpenAI. Esto permite integrar el modelo GPT de OpenAI en la aplicación con acceso controlado a la clave de API.

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

El código carga un prompt desde LangChain Hub utilizando hub.pull("rlm/rag-prompt"). Luego, invoca el prompt con un contexto y una pregunta de ejemplo, y convierte el resultado en mensajes con .to_messages(). Los mensajes generados son almacenados en example_messages, mostrando cómo el prompt procesó la entrada.

In [18]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

El código print(example_messages[0].content) imprime el contenido del primer mensaje generado en example_messages. Esto permite ver la salida específica que se obtuvo después de invocar el prompt con el contexto y la pregunta de ejemplo.

In [19]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


El código configura un flujo de trabajo RAG que obtiene documentos relevantes, los formatea, y luego los pasa a través de un prompt y un modelo de lenguaje (llm). La salida se parsea usando StrOutputParser. Finalmente, imprime los resultados de forma incremental usando stream, mostrando la respuesta a la pregunta "What is Task Decomposition?".

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task Decomposition is the process of breaking down a complex task into smaller, manageable steps to facilitate easier execution and understanding. This technique often involves prompting models to think step by step, allowing them to generate subgoals or outlines for achieving a larger objective. Methods like Chain of Thought and Tree of Thoughts enhance this process by exploring various reasoning possibilities at each step.

El código define una cadena de procesamiento chain que utiliza RAG para obtener el contexto mediante un retriever y formatearlo con format_docs. La pregunta se pasa sin modificaciones usando RunnablePassthrough(). Luego, ambos se envían a través de un prompt para generar una respuesta basada en ese contexto.

In [ ]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

# Built-in chains

El código define una cadena de procesamiento chain que utiliza RAG para obtener el contexto mediante un retriever y formatearlo con format_docs. La pregunta se pasa sin modificaciones usando RunnablePassthrough(). Luego, ambos se envían a través de un prompt para generar una respuesta basada en ese contexto.

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down a complicated task into smaller, more manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance this process by allowing models to think step by step and explore multiple reasoning possibilities. This approach helps improve performance on complex tasks by simplifying them and providing clarity on the model's reasoning.


El código recorre los documentos en el contexto de la respuesta RAG. Imprime el contenido de cada documento, seguido de un salto de línea. Esto permite ver cada fragmento de contexto utilizado para generar la respuesta.

In [22]:
for document in response["context"]:
    print(document)
    print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS 

El código define un prompt personalizado con instrucciones para generar respuestas concisas y agregar "thanks for asking!" al final. Luego, configura una cadena RAG que obtiene el contexto mediante un retriever, lo formatea, y lo envía junto con la pregunta al modelo (llm). Finalmente, invoca la cadena con la pregunta "What is Task Decomposition?" para generar la respuesta

In [23]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is the process of breaking down a complex task into smaller, manageable steps to facilitate planning and execution. Techniques like Chain of Thought (CoT) and Tree of Thoughts allow models to structure their reasoning and explore multiple possibilities for solving problems. This approach enhances model performance and provides insight into the model's thinking process. Thanks for asking!"